In [7]:
!pip install pyTigerGraph
!pip install -q jupyter-dash
!pip install python-dotenv

In [8]:
# Imports
import pandas as pd
import numpy as np
import pyTigerGraph as tg
import dash
from jupyter_dash import JupyterDash
from dash import html

### Open Connection to Graph

In [ ]:
# Connection Params
hostname = 'https://leaguerecommender.i.tgcloud.io/'
username = 'tigergraph'
password = 'tigergraph'
graphname = 'Top3ChampionRecommender'

In [ ]:
conn = tg.TigerGraphConnection(host=hostname, username=username, password=password, graphname=graphname)
conn.apiToken = conn.getToken(conn.createSecret())

print("Connected!")

### Create Dashboard

In [6]:
app = JupyterDash(__name__)

app.layout = html.Div(children=[
    html.P(children='Hello Dash'),

])

app.run_server(mode='external')

 * Tip: There are .env or .flaskenv files present. Do "pip install python-dotenv" to use them.


Dash app running on http://127.0.0.1:8050/
